In [1]:
import os
os.chdir('..')

In [2]:
from model.network import Converter
import matplotlib.pyplot as plt

wandb: WARNING import wandb.keras called before import keras or import tensorflow.keras.  This can lead to a version mismatch, W&B now assumes tensorflow.keras
Using TensorFlow backend.


In [3]:
plt.rcParams["figure.figsize"] = (20,20)

In [4]:
import os
import pickle
from pathlib import Path

import numpy as np
import tensorflow as tf
import lpips_tf

from keras import backend as K
from keras import optimizers, losses, regularizers
from keras.layers import Conv2D, Dense, UpSampling2D, LeakyReLU, Activation
from keras.layers import Layer, Input, Reshape, Lambda, Flatten, Concatenate, Embedding, GaussianNoise
from keras.models import Model, load_model
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, Callback
from keras.applications import vgg16
from keras_lr_multiplier import LRMultiplier
from assets import AssetManager

from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks.callbacks import EarlyStopping, CSVLogger
from keras.callbacks.tensorboard_v1 import TensorBoard

from sklearn.preprocessing import OneHotEncoder
from content_classification import LORDContentClassifier

In [5]:
# class LORDContentClassifier:
#     def __init__(self, subset=None,
#                  base_dir = 'results', model_name = 'minst_10_model', data_name = 'minst_10_test', include_encoders=True):
#         assets = AssetManager(base_dir)
#         data = np.load(assets.get_preprocess_file_path(data_name))
#         imgs, self.classes, self.contents, n_classes = data['imgs'], data['classes'], data['contents'], data['n_classes']
#         imgs = imgs.astype(np.float32) / 255.0
#         if subset is not None:
#             self.curr_imgs = imgs[:subset]
#             self.classes = self.classes[:subset]
#         else:
#             self.curr_imgs = imgs
#             self.classes = self.classes

#         self.onehot_enc = OneHotEncoder()
#         self.onehot_classes = self.onehot_enc.fit_transform(self.classes.reshape(-1,1))
#         self.n_classes = self.onehot_classes.shape[1]

#         self.n_images = self.curr_imgs.shape[0]
        
#         self.converter = Converter.load( assets.get_model_dir(model_name), include_encoders=include_encoders)
#         self.content_codes = self.converter.content_encoder.predict(self.curr_imgs)
#         class_codes = self.converter.class_encoder.predict(self.curr_imgs)
#         class_adain_params = self.converter.class_modulation.predict(class_codes)
#         self.class_adain_params = class_adain_params.reshape(class_adain_params.shape[0], -1)
        
#     def train_content_classifier(self, n_epochs):        
#         model = self.get_model(self.content_codes.shape[1])        
#         callbacks = [EarlyStopping('val_accuracy', patience=10), CSVLogger('LORDContentClassifier_content.csv'), TensorBoard()]
#         return model.fit(self.content_codes, self.onehot_classes, epochs=n_epochs, validation_split=0.3, callbacks=callbacks)
        
#     def train_class_classifier(self, n_epochs):        
#         model = self.get_model(self.class_adain_params.shape[1])        
#         callbacks = [EarlyStopping('val_accuracy', patience=10), CSVLogger('LORDContentClassifier_class.csv'), TensorBoard()]
#         return model.fit(self.class_adain_params, self.onehot_classes, epochs=n_epochs, validation_split=0.3, callbacks=callbacks)
        
#     def get_model(self, input_dim):
#         model = Sequential()
#         model.add(Dense(units=256, activation='relu', input_dim=input_dim))
#         model.add(Dense(units=self.n_classes, activation='softmax'))

#         model.compile(loss='categorical_crossentropy',
#                       optimizer='adam',
#                       metrics=['accuracy'])
#         return model

In [6]:
cc = LORDContentClassifier( 100, model_name='smallnorb_no_adain', data_name = 'smallnorb_strict_class_fxd_train')
cc.train_content_classifier(1000)
# cc.train_class_classifier(10000)

[16 16 16 ...  9  9  9]
(38880,) (38880, 64, 64, 1)
(100,) (100, 64, 64, 1)
loading models...
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '



vgg arch:
Model: "vgg"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64, 64, 1)         0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 64, 64, 3)         0         
_________________________________________________________________
vgg_normalization_1 (VggNorm (None, 64, 64, 3)         0         
_________________________________________________________________
model_1 (Model)              [(None, 64, 64, 64), (Non 14714688  
Total params: 14,714,688
Trainable params: 14,714,688
Non-trainable params: 0
_________________________________________________________________

(100,) (100, 64, 64, 1)


ValueError: You are passing a target array of shape (100, 1) while using as loss `categorical_crossentropy`. `categorical_crossentropy` expects targets to be binary matrices (1s and 0s) of shape (samples, classes). If your targets are integer classes, you can convert them to the expected format via:
```
from keras.utils import to_categorical
y_binary = to_categorical(y_int)
```

Alternatively, you can use the loss function `sparse_categorical_crossentropy` instead, which does expect integer targets.